## Install Dependencies 

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker import TrainingJobAnalytics
import sagemaker as sage
import pandas as pd

In [ ]:
sess = sagemaker.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [ ]:
role = "arn:aws:iam::224041885527:role/dev-lambda-test_sagemaker"
prefix = 'ann-churn'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

In [ ]:
classifier = sage.estimator.Estimator(
                image, 
                role, 
                train_instance_type='ml.m5.large', 
                train_instance_count=1,
                output_path="s3://ml-sagemaker-bucket/output",
                sagemaker_session=sess)

In [ ]:
classifier.fit("s3://ml-sagemaker-bucket/data/breast-cancer-wisconsin.data.csv")